In [ ]:
#importing libraries and reading data
import os
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn  as sns
from sklearn.preprocessing import scale
from datetime import datetime
pal = sns.color_palette()
pd.options.display.max_colwidth = 100

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score, precision_score, recall_score

import keras
from keras.layers import Input
from keras import Model
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.callbacks import Callback
from keras import backend as K

import nltk
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation

import re
import os
import gc
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from textblob import Word
from nltk.corpus import stopwords

print('# File sizes')
for f in os.listdir('../input'):
    if 'zip' not in f:
        print(f.ljust(30) + str(round(os.path.getsize('../input/' + f) / 1000000, 2)) + 'MB')
        
print(os.listdir("../input"))
        
time_record = {}
time_record['entire_script'] = {'start': datetime.now()}

sample_size = 0
submission = True

In [ ]:
# reading trainng data
df_train = pd.read_csv('../input/train.csv')
df_train.head()
if submission:
    df_test = pd.read_csv('../input/test.csv')

In [ ]:
print('Total number of question for training: {}'.format(len(df_train)))
print('Insincere questions: {}%'.format(round(df_train['target'].mean()*100, 2)))

In [ ]:
#dropping top n rows from insincere_df, leaving 6000 qs
if sample_size > 0:
    df_train.drop(df_train.head(len(df_train)-sample_size).index,inplace=True)
    if submission:
        df_test.drop(df_test.head(len(df_test)-sample_size).index,inplace=True)
print(len(df_train))

In [ ]:
print('Total number of question for training: {}'.format(len(df_train)))
print('Insincere questions: {}%'.format(round(df_train['target'].mean()*100, 2)))
insincere = df_train.loc[df_train.target == 1, 'target'].count()
sincere = df_train.loc[df_train.target == 0, 'target'].count()
print("Total number of insincere questions: ", insincere)
print("Total number of sincere questions: ", sincere)

In [ ]:
# preprocessing
features = ['word_IDF_norm']
def preprocessing(df_train):
    start = datetime.now()
    # converting to lowercase
    df_train['question_text'] = df_train['question_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    print('elapsed time1: ',(datetime.now()-start).total_seconds())
    start = datetime.now()

    #removing punctuation
    df_train['question_text'] = df_train['question_text'].str.replace('[^\w\s]','')
    print('elapsed time2: ',(datetime.now()-start).total_seconds())
    start = datetime.now()

    #removing stopwords
    stop = stopwords.words('english')
    df_train['question_text'] = df_train['question_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    print('elapsed time3: ',(datetime.now()-start).total_seconds())
    start = datetime.now()

    #lemmatization
    df_train['question_text'] = df_train['question_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
    print('elapsed time4: ',(datetime.now()-start).total_seconds())
    start = datetime.now()

    #word IDF
    word_list = []
    df_train['question_text'].apply(lambda x: word_list.extend(list(set(x.split()))))
    print('elapsed time5: ',(datetime.now()-start).total_seconds())
    start = datetime.now()
    word_pd = pd.Series(word_list)
    word_row_freq = dict(word_pd.value_counts())
    print('elapsed time8: ',(datetime.now()-start).total_seconds())
    start = datetime.now()

    #start calculating statistics
    df_train['word_row_freq'] = df_train['question_text'].apply(lambda x: [word_row_freq[x] for x in x.split()])
    print('elapsed time9: ',(datetime.now()-start).total_seconds())
    start = datetime.now()
    df_train['word_IDF'] = df_train['question_text'].apply(lambda x: [np.log(df_train.shape[0]/word_row_freq[x]) for x in x.split()])
    print('elapsed time10: ',(datetime.now()-start).total_seconds())
    start = datetime.now()
    idf_data = []
    df_train['word_IDF'].apply(lambda x: idf_data.extend(x))
    print('elapsed time11: ',(datetime.now()-start).total_seconds())
    start = datetime.now()
    idf_mean = np.mean(idf_data)
    idf_var = np.var(idf_data)
    print('elapsed time12: ',(datetime.now()-start).total_seconds())
    start = datetime.now()
    df_train['word_IDF_norm'] = df_train['word_IDF'].apply(lambda x: np.divide(np.subtract(x,idf_mean),idf_var))
    print('elapsed time13: ',(datetime.now()-start).total_seconds())
    start = datetime.now()
    df_train.drop(columns=['word_row_freq','word_IDF'], inplace=True)
    print('elapsed time14: ',(datetime.now()-start).total_seconds())

    df_train.head()
    return df_train
    
time_record['preprocessing'] = {'start': datetime.now()}
df_train = preprocessing(df_train)
if submission:
    df_test = preprocessing(df_test)
time_record['preprocessing']['end'] = datetime.now()

In [ ]:
train, val  = train_test_split(df_train, test_size = 0.1, random_state = 13)
if submission:
    test = df_test

In [ ]:
#total unique words
time_record['unique_words'] = {'start': datetime.now()}
def count_unique_words(pandasdf):
    pandasdf.str.lower().str.split()
    results = set()
    pandasdf.str.lower().str.split().apply(results.update)
    uniquewords = len(results)
    #print(uniquewords)
    return uniquewords

# maximum question length
def max_question_len(pandasdf):
    return max(pandasdf.str.split(" ").str.len())

SERIES_LENGTH = max_question_len(train['question_text'])
NUM_UNIQUE_WORDS = count_unique_words(train['question_text'])
print("Series Length: {}\nNumber of unique words:{}".format(SERIES_LENGTH, NUM_UNIQUE_WORDS))
time_record['unique_words']['end'] = datetime.now()

In [ ]:
time_record['tokenization_padding'] = {'start': datetime.now()}

tokenizer = Tokenizer(num_words=NUM_UNIQUE_WORDS)
tokenizer.fit_on_texts(list(train['question_text']))
    
temp = tokenizer.texts_to_sequences(train['question_text'])
train['question_text_padded'] = sequence.pad_sequences(temp, maxlen=SERIES_LENGTH, padding='pre').tolist()
for feature in features:
    train[feature+"_padded"] = sequence.pad_sequences([i for i in train[feature]], maxlen=SERIES_LENGTH, padding='pre', dtype='float32').tolist()
    
temp = tokenizer.texts_to_sequences(val['question_text'])
val['question_text_padded'] = sequence.pad_sequences(temp, maxlen=SERIES_LENGTH, padding='pre').tolist()
for feature in features:
    val[feature+"_padded"] = sequence.pad_sequences([i for i in val[feature]], maxlen=SERIES_LENGTH, padding='pre', dtype='float32').tolist()
    
if submission:
    temp = tokenizer.texts_to_sequences(test['question_text'])
    test['question_text_padded'] = sequence.pad_sequences(temp, maxlen=SERIES_LENGTH, padding='pre').tolist()
    for feature in features:
        test[feature+"_padded"] = sequence.pad_sequences([i for i in test[feature]], maxlen=SERIES_LENGTH, padding='pre', dtype='float32').tolist()

time_record['tokenization_padding']['end'] = datetime.now()

In [ ]:
# glove embeddinng, from starter code 
time_record['embedding'] = {'start': datetime.now()}

def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
    
def get_embed_mat(EMBEDDING_FILE, max_features,embed_dim):
    # word vectors
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding='utf8'))
    print('Found %s word vectors.' % len(embeddings_index))

    # embedding matrix
    word_index = tokenizer.word_index
    num_words = min(max_features, len(word_index) + 1)
    all_embs = np.stack(embeddings_index.values()) #for random init
    embedding_matrix = np.zeros((len(word_index) + 1, embed_dim))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    max_features = embedding_matrix.shape[0]
    
    return embedding_matrix

gloveEmbed = get_embed_mat('../input/embeddings/glove.840B.300d/glove.840B.300d.txt', NUM_UNIQUE_WORDS, 300)
embedding_layer = Embedding(len(tokenizer.word_index)+1,300,weights=[gloveEmbed],input_length=SERIES_LENGTH,trainable=False)
time_record['embedding']['end'] = datetime.now()

In [ ]:
time_record['augmentation'] = {'start': datetime.now()}
def data_augmentation(df_train):
    insincere_df = df_train.loc[df_train['target'] == 1]
    sincere_df = df_train.loc[df_train['target'] == 0]
    insincere1 = insincere_df
    insincere2 = insincere_df
    insincere3 = insincere_df
    insincere4 = insincere_df
    insincere5 = insincere_df
    fake_insincere = pd.concat([insincere1, insincere2, insincere3, insincere4, insincere5, insincere1, insincere2, insincere3, insincere4, insincere5, insincere1, insincere2, insincere3, insincere4, insincere5], ignore_index=True)
    df_quora = pd.concat([sincere_df, fake_insincere], ignore_index=True)
    return df_quora

train_aug = data_augmentation(train)
#basic data info
print('Total number of question for training: {}'.format(len(train_aug)))
print('Insincere questions: {}%'.format(round(train_aug['target'].mean()*100, 2)))
time_record['augmentation']['end'] = datetime.now()

In [ ]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
time_record['training'] = {'start': datetime.now()}

def make_model_b():
    feature_num = 1
    lstm_out = 100
    features = Input(shape=(SERIES_LENGTH, feature_num), dtype='float32', name='features')
    word_tokens = Input(shape=(SERIES_LENGTH,), dtype='float32', name='word_tokens')
    embedded_words = embedding_layer(word_tokens)
    conc = keras.layers.concatenate([embedded_words, features])
    lstm = LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.1)(conc)
    dens = Dense(1,activation='sigmoid', name='output')(lstm)
    
    model = Model([word_tokens, features], dens)
    model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = [f1])
    print(model.summary())
    
    return model

Y_train = train_aug['target'].values
X_train_word_tokens = np.array([i for i in train_aug['question_text_padded']])
X_train_features = np.array([i for i in train_aug['word_IDF_norm_padded']])

X_val_word_tokens = np.array([i for i in val['question_text_padded']])
X_val_features = np.array([i for i in val['word_IDF_norm_padded']])
Y_val = val['target'].values
    
X_train_features = np.expand_dims(X_train_features, axis=2)
X_val_features = np.expand_dims(X_val_features, axis=2)

if submission:
    X_test_word_tokens = np.array([i for i in test['question_text_padded']])
    X_test_features = np.array([i for i in test['word_IDF_norm_padded']])
    X_test_features = np.expand_dims(X_test_features, axis=2)

print(X_train_word_tokens.shape, X_train_features.shape, Y_train.shape)
print(X_val_word_tokens.shape, X_val_features.shape, Y_val.shape)

model = make_model_b()
epoch_size = 5
if not submission:
    history = model.fit([X_train_word_tokens, X_train_features],
                    Y_train, batch_size=512,epochs=epoch_size,verbose=1,
                    validation_data=[[X_val_word_tokens, X_val_features], Y_val])
else:
    history = model.fit([X_train_word_tokens, X_train_features],
                    Y_train, batch_size=512,epochs=epoch_size,verbose=1)

time_record['training']['end'] = datetime.now()

In [ ]:
if not submission:
    plt.plot(history.history['val_f1'])
plt.plot(history.history['f1'])
for stat in history.history:
    print(stat,history.history[stat])

In [ ]:
def line_search_f1_score(y_score, y_test):
    max_f1_score = 0
    opt_threshold = 0
    for threshold in [i*0.01 for i in range(100)]:
        y_preds = y_score > threshold
        score = f1_score(y_preds, y_test)
        if max_f1_score < score:
            max_f1_score = score
            opt_threshold = threshold
    return max_f1_score, opt_threshold

time_record['threshold'] = {'start': datetime.now()}

y_score_val = model.predict([X_val_word_tokens, X_val_features], verbose=1)
max_f1_score, threshold = line_search_f1_score(y_score_val, Y_val)
    
print(threshold)
print(max_f1_score)

time_record['threshold']['end'] = datetime.now()

In [ ]:
if submission:
    time_record['submission'] = {'start': datetime.now()}

    y_score_test = model.predict([X_test_word_tokens, X_test_features], verbose=1)

    df_test['prediction'] = np.array(y_score_test > threshold, dtype=int)
    df_test[['qid','prediction']].to_csv("submission.csv", index=False)

    time_record['submission']['end'] = datetime.now()
time_record['entire_script']['end'] = datetime.now()

In [ ]:
for subsection in time_record:
    print("{}: {}".format(subsection, (time_record[subsection]['end']-time_record[subsection]['start']).total_seconds()))